<a href="https://colab.research.google.com/github/Noznoc/access-classifier/blob/main/WIP_Exterior_Accessibility_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN Binary Classifier Building Enterances
**Author:** Alex Lane and Julia Conzon <br>
**Date created:** 2021/03/18<br>
**Description:** Training an image classifier from scratch on the Mapillary API imagery collected on building enterances. This workflow follows the Classifier from [Scratch Keras tutorial](https://keras.io/examples/vision/image_classification_from_scratch/).

## Setup


Import the necessary Python packages for evaluation.

In [8]:
import sys # for importing directory
import tensorflow as tf # for modeling
from tensorflow import keras # for modeling
from keras.models import Sequential # following Wu et al. (2019) sequential model was used
from tensorflow.keras import layers
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, Activation, BatchNormalization
import matplotlib.pyplot as plt # for visualizing images
import numpy as np

## Load the data: "Accessible" and "Inaccessible" dataset

Compile data from the Mapillary API, [scripts accessible in another notebook]()

Confirm folder of Mapillary API imagery exists. Subfolders should be `Accessible` and `Inaccessible`, which each contain the images that represent the category.


In [ ]:
!ls OSM_And_Mapillary_Sequence_Dataset

import os, random
import shutil

# Function that randomly moves image files based on source directory, destination directory and number of files wanting to move
def randomly_move_imgs(src_dir, dst_dir, n):
  file_list = os.listdir(src_dir)
  for j in range(n):
    img = random.choice(file_list)
    shutil.move(src_dir + img, dst_dir)

# Read "OSM_And_Mapillary_Sequence_Dataset/Accessible" folder randomly select n and move to "Test/Accessible" folder
n = 10
src_dir = "./OSM_And_Mapillary_Sequence_Dataset/Train/Accessible/"
dst_dir = "./OSM_And_Mapillary_Sequence_Dataset/Test/Accessible"
# randomly_move_imgs(src_dir, dst_dir, n)

# Read "OSM_And_Mapillary_Sequence_Dataset/Inaccessible" folder and randomly select n and move to "Test/Inaccessible" folder
n = 10
src_dir = "./OSM_And_Mapillary_Sequence_Dataset/Train/Inaccessible/"
dst_dir = "./OSM_And_Mapillary_Sequence_Dataset/Test/Inaccessible"
# randomly_move_imgs(src_dir, dst_dir, n)

Model  Test  Train


## Preprocess Data


Steps includes standardizing the images to the same size (`image_size`). 

Keras `seed` command is an optional parameter to offer a random seed for shuffling and transforming the images to augment the dataset. 

In [9]:
image_size = (512, 512)
batch_size = 32


# Assign to a train dataset (85%)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "OSM_And_Mapillary_Sequence_Dataset/Train",
    validation_split=0.15,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode = 'binary',
)

# Assign to a validation dataset (15%)
valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "OSM_And_Mapillary_Sequence_Dataset/Train",
    validation_split=0.15,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode = 'binary',
)

# confirm classes
class_names = train_ds.class_names
print(class_names)

Found 824 files belonging to 2 classes.
Using 701 files for training.
Found 824 files belonging to 2 classes.
Using 123 files for validation.
['Accessible', 'Inaccessible']


## Visualize Data

Label 1 is "inaccessible" and label 0 is "accessible".


In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")


## Augmentation

Considering the small sample for training, the dataset will be augmented, that is randomly flipped horizontally, rotated, contrasted and zoomed. Order impacts the image gets distorted. Different tests were applied to identify an optimal augmentation, as seen below:

https://www.pyimagesearch.com/2019/07/08/keras-imagedatagenerator-and-data-augmentation/


In [10]:
data_augmentation = keras.Sequential(
    [
        # Reminder, the order in which transformations are called significantly 
        # affects the final augmentation.
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomTranslation(0.1,0.1),
        layers.experimental.preprocessing.RandomZoom(0.2),
        layers.experimental.preprocessing.RandomRotation(0.1),
        layers.experimental.preprocessing.RandomContrast(0.01),
        #layers.experimental.preprocessing.RandomHeight(0.5),
        #layers.experimental.preprocessing.RandomWidth(0.5),
        #layers.experimental.preprocessing.Resizing(180, 180),
    ]
)

Visualize the augmented samples:


In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")


## Configure Dataset

Following documentation from Keras creator, François Chollet, use buffered prefetching to yield data from disk without having I/O becoming blocking:


In [11]:
train_ds = train_ds.prefetch(buffer_size=32)
valid_ds = valid_ds.prefetch(buffer_size=32)


## Build Our Model





In [ ]:
# MODEL ARCHITECTURES BY WU ET AL (2019):

# CNN2: CONV(64, 3), RELU, CONV(32, 3), RELU, POOL(2), CONV(64, 3), FC(128), RELU, FC(2), SOFTMAX.
# CNN3: [CONV(128, 3), RELU, POOL(2)], [CONV(64 3), RELU, POOL(2)], [CONV(32, 3), RELU, POOL(2)], DROPOUT(), RELU, FC(512), RELU, FC(2), SOFTMAX.
# CNN5: CONV(128, 3), RELU, CONV(128, 3), RELU, POOL(2), CONV(64, 3), DROPOUT(), RELU, CONV(64, 3), RELU, POOL(2), CONV(32, 3), DROPOUT(), RELU, POOL(2), FC(512), RELU, DROPOUT(), FC(256), RELU, FC(2), SOFTMAX

# Model parameters
input_shape = image_size + (3,) # add RGB code of 3 to image shape for Keras Input() argument
num_classes = 2 # number of classes being defined

# Initialize model
inputs = keras.Input(shape=input_shape) # establish inputs e.g., (180, 180, 3) is the width and height dimension and 3 represents RGB
print(inputs)
layer = data_augmentation(inputs) # first "layer" is augmented images that are called in the previous script

# rescale the image to 1 and 0, not 255 RGB
layer = layers.experimental.preprocessing.Rescaling(1.0 / 255)(layer)

# Building blocks for CNN

layer = Conv2D(filters=16, kernel_size=3)(layer)
layer = Activation('relu')(layer)
layer = MaxPooling2D(pool_size = (2,2))(layer)

layer = Conv2D(filters=32, kernel_size=3, activation='relu')(layer)
layer = MaxPooling2D(pool_size = (2,2))(layer)

layer = Conv2D(filters=64, kernel_size=3, activation='relu')(layer)
layer = MaxPooling2D(pool_size = (2,2))(layer)

layer = Conv2D(filters=64, kernel_size=3, activation='relu')(layer)
layer = MaxPooling2D(pool_size = (2,2))(layer)

layer = Conv2D(filters=128, kernel_size=3, activation='relu')(layer)
layer = MaxPooling2D(pool_size = (2,2))(layer)

layer = Flatten()(layer)

layer = Dense(512, activation='relu')(layer)
outputs = Dense(1, activation='sigmoid')(layer) # or use sigmoid to view probability distribution

print(outputs)

# Initialize model with building blocks
model = keras.Model(inputs, outputs)

# Plot model architecture
keras.utils.plot_model(model, to_file='model.png', rankdir='TR', show_shapes='True')

# Print summary of model
# model.summary()

# Print each layer
# for layer in model.layers:
#   print(layer.output_shape)

# Build Wu CNN2 Model - WILL CRASH
Start off with a simple Sequential model, following Wu et al. (2019)
Their CNNs require way too much memory, OutOfMemoryError, we'd need a better GPU

In [ ]:
# MODEL ARCHITECTURES BY WU ET AL (2019):

# CNN2: CONV(64, 3), RELU, CONV(32, 3), RELU, POOL(2), CONV(64, 3), FC(128), RELU, FC(2), SOFTMAX.
# CNN3: [CONV(128, 3), RELU, POOL(2)], [CONV(64 3), RELU, POOL(2)], [CONV(32, 3), RELU, POOL(2)], DROPOUT(), RELU, FC(512), RELU, FC(2), SOFTMAX.
# CNN5: CONV(128, 3), RELU, CONV(128, 3), RELU, POOL(2), CONV(64, 3), DROPOUT(), RELU, CONV(64, 3), RELU, POOL(2), CONV(32, 3), DROPOUT(), RELU, POOL(2), FC(512), RELU, DROPOUT(), FC(256), RELU, FC(2), SOFTMAX

# Model parameters
input_shape = image_size + (3,) # add RGB code of 3 to image shape for Keras Input() argument
num_classes = 2 # number of classes being defined

# Initialize model
inputs = keras.Input(shape=input_shape) # establish inputs e.g., (180, 180, 3) is the width and height dimension and 3 represents RGB
print(inputs)
layer = data_augmentation(inputs) # first "layer" is augmented images that are called in the previous script

# rescale the image to 1 and 0, not 255 RGB
layer = layers.experimental.preprocessing.Rescaling(1.0 / 255)(layer)

# Building blocks for CNN

layer = Conv2D(filters=64, kernel_size=3, activation='relu')(layer)
layer = MaxPooling2D(pool_size = (2,2))(layer)

layer = Conv2D(filters=32, kernel_size=3, activation='relu')(layer)
layer = MaxPooling2D(pool_size = (2,2))(layer)

layer = Flatten()(layer)

layer = Dense(128, activation='relu')(layer)

outputs = Dense(2, activation='softmax')(layer) # or use sigmoid to view probability distribution

print(outputs)

# Initialize model with building blocks
model = keras.Model(inputs, outputs)

# Plot model architecture
keras.utils.plot_model(model, to_file='model.png', rankdir='TR', show_shapes='True')

# Print summary of model
# model.summary()

# Print each layer
# for layer in model.layers:
#   print(layer.output_shape)

# Train Model

In [ ]:
# Number of epochs
epochs = 50

# Add callbacks to log file
callbacks = [
    keras.callbacks.ModelCheckpoint("OSM_And_Mapillary_Sequence_Data/Model/Epochs/{epoch}.h5"),
]

# Compile CNN, following default optimizer and loss function for binary classification
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

classifier = model.fit(train_ds, epochs=epochs, callbacks=callbacks, validation_data=valid_ds, shuffle=True)

# This following visualization is taken from 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/
# summarize history for accuracy
plt.plot(classifier.history['accuracy'])
plt.plot(classifier.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(classifier.history['loss'])
plt.plot(classifier.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# Save Model

In [ ]:
model.save('./OSM_And_Mapillary_Sequence_Dataset/Model/Saved Models/50 Epoch Low Val Acc/')

# Load Model

Alternatively, load a model instead of training a new one

In [32]:
model = keras.models.load_model('./OSM_And_Mapillary_Sequence_Dataset/Model/Saved Models/100 Epoch Overfit/')

# Test Model

I think the scores were opposite, so I changed the code somewhat.

In [ ]:
import os

directory = r'./OSM_And_Mapillary_Sequence_Dataset/Test/Accessible/'
correctInaccessible = 0
correctAccessible = 0

for testImage in os.listdir(directory):
    if testImage.endswith(".png"):
        #print(os.path.join(directory, testImage))
        img = keras.preprocessing.image.load_img(os.path.join(directory, testImage), 
                                                 target_size=image_size)
        img_array = keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)  # create batch axis

        predictions = model.predict(img_array)
        score = predictions[0]
        plt.imshow(img)
        plt.axis("off")
        plt.pause(0.1)
        print(testImage)
        print(
            "This image of building entrance is %.2f percent accessible and %.2f percent inaccessible."
            % (100 * (1 - score), 100 * score)
        )
        if((1 - score) > 0.5):
            print("✅ - This image has been correctly classified as accessible.\n")
            correctAccessible += 1
        elif ((1 - score) == 0.5):
            print("❓ - The model say this image has equal probability of being accesible or inaccessible.\n")
        else:
            print("❌ - This image has been incorrectly classified as inaccessible.\n")

directory = r'./OSM_And_Mapillary_Sequence_Dataset/Test/Inaccessible/'
for testImage in os.listdir(directory):
    if testImage.endswith(".png"):
        #print(os.path.join(directory, testImage))
        img = keras.preprocessing.image.load_img(os.path.join(directory, testImage), 
                                                 target_size=image_size)
        img_array = keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)  # create batch axis

        predictions = model.predict(img_array)
        score = predictions[0]
        plt.imshow(img)
        plt.axis("off")
        plt.pause(0.1)
        print(testImage)
        print(
            "This image of building entrance is %.2f percent accessible and %.2f percent inaccessible."
            % (100 * (1 - score), 100 * score)
        )
        if(score > 0.5):
            print("✅ - This image has been correctly classified as inaccessible.\n")
            correctInaccessible += 1
        elif (score == 0.5):
            print("❓ - The model say this image has equal probability of being accesible or inaccessible.\n")
        else:
            print("❌ - This image has been incorrectly classified as accessible.\n")

print("%i of the 10 accessible images were correctly classified as accessible." % correctAccessible)
print("%i of the 10 inaccessible images were correctly classified as inaccessible." % correctInaccessible)
